# **Empowering Healthcare with Symbolic Learning and Knowledge Graph Embeddings**

![Desing Pattern](https://github.com/SDM-TIB/HyAI/assets/25593410/42cf771d-b82d-4097-b0f7-0ceb3581b171)

## **Pattern Detection, Analysis and Explanation over P4-LUCAT Knowledge Graph**

>> **Pattern Detection. Unsupervised learning**
*   Learned embeddings of the entities in the Knowledge Graph
*   Apply community detection Algorithms: SemEP, KMeans
*   Compute the quality of the communities generated
*   Generate Radar-plots 
*   Visualize PCA Projection to 2D of the detected cummunities


Git clone

In [ ]:
!git clone https://github.com/SDM-TIB/SymbolicLearning_KGE.git
%cd SymbolicLearning_KGE/PatternDetection

In [ ]:
import ComputeCommunities as SemCD
import EvaluationMetric

**Clean the build**

In [ ]:
!make clean

**Build the project**

In [ ]:
!make

## Run the semEP-node

**Parameter input**

In [ ]:
model_list = ['TransH', 'RotatE']
threshold = [30, 40, 50, 60, 70, 80]
kg_name = 'TransformedKG'
target_predicate = 'hasRelapse_Progression'

In [ ]:
SemCD.run(kg_name=kg_name, target_predicate=target_predicate,
          model_list=model_list, threshold=threshold)

## Quality of the generated communities 

Moving to clusteringMeasures folder 

In [ ]:
%cd clusteringMeasures

**Clean the build**

In [ ]:
!make clean

**Build the project**

In [ ]:
!make

**Compute Metrices**

In [ ]:
import ComputeMetrices
ComputeMetrices.run(model_list, threshold)

### Generate Radar Plot

In [ ]:
%cd ../
EvaluationMetric.GenerateRadarPlot(model_list, threshold)